# Relatório final do trabalho de grafos

Alunos: Liliana Sabato Teodoro (201810021)

       Rafaela Custorio (201720376)

       Ruan Basilio (201720089)

Turma: 14

Professor: Mayron César de O. Moreira

# Introdução

Um dos problemas estudados em grafos é o problema da coloração de vértices que consiste em atribuir cores (rótulos) aos 
vértices do grafo de maneira a não se ter vértices ligados com cores iguais, o problema de otimização da coloração é 
colorir o grafo com o menor número de cores possível, afim de se encontrar a solução ótima da coloração. Uma das variações desse problema está relacionado à montagem de horários escolares utilizando abordagem de coloração em grafos, como a montagem desses horários pode conter diversas restrições como restrições do próprio MEC (Ministério da Educação), da escola e dos professores, faz com que seja difícil obter soluções ótimas em todos os casos. 

# Objetivo

O objetivo desse trabalho é tentar encontrar uma solução para a criação de horários de determinadas escolas, utilizando abordagem em
grafos e coloração de vértices, atendendo ao menos as restrições impostas pela escola e professores, visando também atender os pedidos
de horários dos professores.

# Desenvolvimento

Para o desenvolvimento do trabalho foi utilizado a linguagem Python, mais precisamente a versão 3.6.9, com uma abordagem utilizando coloração de grafos, para implementação do 
grafo foi usado uma estrutura de lista de adjacência em que o grafo é não direcionado, classes auxiliares foram utilizadas
afim de ajudar a visualização e implementação no geral. Segue abaixo as classes auxiliare lista onde é chamada pela classe da lista de adjacencia.

In [1]:
class Lista:
    inicio = None
    fim = None
    
    def adicionaVertice(self, ID):
        if(self.inicio == None):
            self.inicio = Noh(ID)
            self.fim = self.inicio
            return

        novo = Noh(ID)
        self.fim.proximo = novo
        self.fim = novo

E complementando a classe Lista tem-se a classe Noh, que segue os padrões de node de algortimos de listas comuns guardando
o próximo item e sua relação. Segue o código da classe Noh.

In [ ]:
class Noh:
    ID = None
    proximo = None
    relacao = None
    def __init__(self, ID):
        self.ID = ID

Para modelagem em grafos, foi necessário adotar uma estratégia em relação a como seria os vértices do grafo e suas arestas,
a modelagem do vértice consiste em pegar as informações de uma planilha e tratar os dados para construir os vértices e suas
arestas. O vértice é composto pelo id de identificação criado pelo código em si, as informações adicionais do vértice são 
coletadas fazendo uma relação do professor, a quantidade de aula e a turma em que será lecionada a matéria. Se um professor
leciona a mesma matéria mais de uma vez é criado vértices diferentes, sendo usado a diferenciação pelo id. Todo vértice por padrão é criado sem cor, o que é simbolizado pelo número negativo -1. As ligações dos 
vértices será tratado mais a frente. Segue o código da classe Vertice.

In [3]:
class Vertice:
    ID = None
    professor = None
    materia = None
    turma = None
    cor = None
    grau = None
    restricoes = None
    preferencias = None
    preferenciasNaoAtendidas = None

    def __init__(self, ID, prof, mat, Turma):
        self.ID = ID
        self.professor = prof
        self.materia = mat
        self.turma = Turma
        self.cor = -1
        self.grau = 0
        self.restricoes = []
        self.preferencias = []
        self.preferenciasNaoAtendidas = 0

Como dito anteriormente uma lista de adjacência foi usada como estrutura de dados para modelagem do grafo, vários atributos
foram usados para a resolução do problema, a implementação mais a fundo dessa estrutura de dados consiste em um dicionário
Python de listas, sendo essa lista a estrutura auxiliar dito mais acima. Segue o código da classe ListaAdjacencia.

In [4]:
class ListaAdjacencia:
    listaAdj = None
    vertices = None
    ID = 0
    totalPreferencias = None
    horarios = None
    preferenciasAtendidas = None
    cores = None
    naoColoridos = None
    segunda = None
    terca = None
    quarta = None
    quinta = None
    sexta = None

    def __init__(self):
        self.listaAdj = {}
        self.vertices = {}
        self.ID = 0
        self.totalPreferencias = 0
        self.horarios = 0
        self.segunda = None
        self.terca = None
        self.quarta = None
        self.quinta = None
        self.sexta = None
        self.preferenciasAtendidas = []
        self.cores = None

Para a criação do vértice é utilizado o id que é um atributo da própria lista de adjacência, e como dito anteriormente, 
faz a relação do professor, da materia e a turma lecionada, lembrando que pode ter vértices com informações iguais porém
o id difere devido ao fato de algumas matérias serem lecionadas mais de uma vez na semana. Segue o código de criação do
vértice.

In [5]:
def adicionaVertice(self, professor, materia, turma):
        v = Vertice(self.ID, professor, materia, turma)
        self.vertices[self.ID] = v
        self.listaAdj[self.ID] = Lista()
        self.ID+=1

As ligações das arestas é feito na classe da lista de adjacência, seŕa explicado mais a frente como as ligações são selecionadas.
Segue a criação de arestas no grafo.

In [6]:
    def adicionaAresta(self, IDV1, IDV2):
        self.listaAdj[IDV1].adicionaVertice(IDV2)

Para se obter os vértices adjacentes de um vértice é utilizado o método de obter os vizinhos.

In [7]:
    def getVizinhos(self, v):
        vizinhos = []
        listaVizinhos = self.listaAdj[v]
        viz = listaVizinhos.inicio
        while(viz != None):
            vizinhos.append(viz.ID)
            viz = viz.proximo
        return vizinhos

Com a obtenção dos vizinhos do vértice é possível ajustar o grau dos vértices.

In [8]:
    def ajustaGrau(self):
        for v in self.listaAdj:
            self.vertices[v].grau = len(self.getVizinhos(v))

Para auxiliar na implementção foi usado um método para retornar o valor do id geral, que corresponde ao número de vértices
que o grafo possui.

In [9]:
    def numeroVertices(self):
        return self.ID

A ligação dos vértices é uma parte muito importante para resolução do problema de agendamento de horários, para a criação
das arestas que ligam os vértices é utilizado o seguinte critério, caso dois vértices possuam o mesmo professor e/ou a mesma
turma, é criada essa aresta, afim de se manter a integridade das restrições de não se ter duas aulas no mesmo horário para
a mesma turma e duas aulas no mesmo horário para o professor. O método retorna um booleando que será tratado na leitura dos
dados dos arquivos.

In [10]:
    def verificaConflito(self, ID1, ID2):
        if(self.vertices[ID1].professor == self.vertices[ID2].professor):
            return True
        elif(self.vertices[ID1].turma == self.vertices[ID2].turma):
            return True
        else:
            return False

No algoritmo de coloração, que será visto mais a frente é necessário obter o vértice inicial para executar o algoritmo, e é
escolhido o vértice de maior grau.

In [11]:
    def getInicial(self):
        maiorGrau = 0
        for v in self.listaAdj:
            grauVertice = len(self.getVizinhos(v))
            if(grauVertice > maiorGrau):
                maiorGrau = grauVertice
                inicial = self.vertices[v]
        return inicial

Na resolução do problema foi pensado em criar uma lista de cores possíveis para colorir o grafo, de forma a se converter 
os possíveis horários em cores para o grafo, a cor seria um número que varia de 0 até o limite de cores que seria possível
colorir o grafo, por exemplo, se uma escola possui 6 horários por dia da semana, então teria no máximo 30 horários disponíveis
ou seja, 30 cores, cada dia da semana com seus respectivos 6 horários. Para isso é feito a relação entre a tabela de horários
e as cores que cada dia pode assumir.

In [12]:
    def criaListaCores(self, configuracao):
        self.horarios = len(configuracao)
        self.segunda = {}
        self.segunda[configuracao[0]] = 0
        for i in range(1, len(configuracao)):
            self.segunda[configuracao[i]] = i * 5
        
        self.terca = {}
        self.terca[configuracao[0]] = 1
        cont = 1
        for i in range(1, len(configuracao)):
            self.terca[configuracao[i]] = cont + 5
            cont+=5
        

        self.quarta = {}
        self.quarta[configuracao[0]] = 2
        cont = 2
        for i in range(1, len(configuracao)):
            self.quarta[configuracao[i]] = cont + 5
            cont+=5

        self.quinta = {}
        self.quinta[configuracao[0]] = 3
        cont = 3
        for i in range(1, len(configuracao)):
            self.quinta[configuracao[i]] = cont + 5
            cont+=5
        
        self.sexta = {}
        self.sexta[configuracao[0]] = 4
        cont = 4
        for i in range(1, len(configuracao)):
            self.sexta[configuracao[i]] = cont + 5
            cont+=5

Após se ter as listas de cores, cada lista simbolizando um dia, foi necessário a criação das restrições impostas pela escola 
e pelos professores, restrições essas que não podem ser violadas, pois inviabilizam a criação dos horários. Tem-se as restrições
do horário do professor, podendo ser horários em que o mesmo leciona em outras escolas, e horários em que a turma não deve 
possuir aulas. Segue a criação desses dois grupos de restrições.

In [14]:
 def restricoesTurma(self, restricaoTurma):
        qtdLinhas = int(restricaoTurma.index.stop)
        for i in range(qtdLinhas):
            turma = restricaoTurma["Turma"][i]
            dia = restricaoTurma["Dia da semana"][i]
            horario = str(restricaoTurma["Horário da restrição"][i])
            for elemento in self.listaAdj:
                if(self.vertices[elemento].turma == turma):
                    if(dia == "Segunda"):
                        self.vertices[elemento].restricoes.append(self.segunda[horario])
                    elif(dia == "Terça"):
                        self.vertices[elemento].restricoes.append(self.terca[horario])
                    elif(dia == "Quarta"):
                        self.vertices[elemento].restricoes.append(self.quarta[horario])
                    elif(dia == "Quinta"):
                        self.vertices[elemento].restricoes.append(self.quinta[horario])
                    elif(dia == "Sexta"):
                        self.vertices[elemento].restricoes.append(self.sexta[horario])
    
    def restricoesProfessor(self, restricaoProfessor):
        qtdLinhas = int(restricaoProfessor.index.stop)
        for i in range(qtdLinhas):
            professor = restricaoProfessor["Professor:"][i]
            dia = restricaoProfessor["Dia da semana da restrição:"][i]
            horario = str(restricaoProfessor["Restrição de Horário:"][i])
            for elemento in self.listaAdj:
                if(self.vertices[elemento].professor == professor):
                    if(dia == "Segunda" and self.segunda[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].restricoes.append(self.segunda[horario])
                    elif(dia == "Terça" and self.terca[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].restricoes.append(self.terca[horario])
                    elif(dia == "Quarta" and self.quarta[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].restricoes.append(self.quarta[horario])
                    elif(dia == "Quinta" and self.quinta[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].restricoes.append(self.quinta[horario])
                    elif(dia == "Sexta" and self.sexta[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].restricoes.append(self.sexta[horario])

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

Como foi citado acima, além das restrições tem-se as preferências dos professores, seriam horários que os mesmos desejam lecionar 
porém essas preferências não são obrigatórias, apenas desejáveis, para uma solução boa as preferências são levadas em consideração
 mas algumas vezes é necessário o não cumprimento das preferências para viabilizar a criação dos horários. Segue a criação das prefêrencias

In [ ]:
 def preferenciasProfessor(self, preferencias):
        qtdLinhas = int(preferencias.index.stop)
        self.totalPreferencias = qtdLinhas
        for i in range(qtdLinhas):
            professor = preferencias["Professor:"][i]
            dia = preferencias["Dia da semana:"][i]
            horario = str(preferencias["Horário:"][i])
            for elemento in self.listaAdj:
                if(self.vertices[elemento].professor == professor):
                    if(dia == "Segunda" and self.segunda[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].preferencias.append(self.segunda[horario])
                    elif(dia == "Terça" and self.terca[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].preferencias.append(self.terca[horario])
                    elif(dia == "Quarta" and self.quarta[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].preferencias.append(self.quarta[horario])
                    elif(dia == "Quinta" and self.quinta[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].preferencias.append(self.quinta[horario])
                    elif(dia == "Sexta" and self.sexta[horario] not in self.vertices[elemento].restricoes):
                        self.vertices[elemento].preferencias .append(self.sexta[horario])
                    self.vertices[elemento].preferenciasNaoAtendidas += 1

Após a execução do algoritmo de coloração, que será visto mais a frente, é preciso fazer duas verificações para a análise 
da solução, a verificação de vértices que não foram coloridos, que ocorre quando o máximo de cores é alcançado e ainda tem 
vértices no grafo sem cor, e a verificação da porcentagem de preferências que foram atendidas, o ideal seria 100%, mas como
citado anteriormente as vezes é necessário deixar de atender as preferências para viabilizar a solução.

In [ ]:
 def verificaColoracao(self):
        cont = 0
        for elemento in self.listaAdj:
            if(self.vertices[elemento].cor == -1):
                cont+=1
        self.naoColoridos = cont
        return cont
    
    def verificaPreferencias(self):
        return (len(self.preferenciasAtendidas)/self.totalPreferencias)

O algoritmo de coloração utilizado na solução foi o algoritmo DSatur, porém com mudanças para ter mais enfâse na solução 
do problema dos horários das escolas, sendo que foi abordado duas versões do algoritmo devido ao fato de ser um algoritmo 
de conversão rápida não se teve problema no quesito de tempo para a execução. A primeira versão é o algoritmo puro, por assim
dizer, nesse caso não é priorizado a coloração da preferência do professor caso ocorra de ser uma cor preferencial é contabilizada
mas o algoritmo não foca nisso, o foco é apenas garantir as restrições.

In [ ]:
def DSATUR(grafo):
    listaAdj = grafo.listaAdj
    vertices = grafo.vertices
    horarios = grafo.horarios
    cores = []
    verticeAtual = grafo.getInicial()
    for v in listaAdj:
        coresVizinhos = []
        for vizinhos in grafo.getVizinhos(verticeAtual.ID):
            if(vertices[vizinhos].cor != -1 and vertices[vizinhos].cor not in coresVizinhos):
                coresVizinhos.append(vertices[vizinhos].cor)
        for i in range (horarios*5):
            if(i not in coresVizinhos and i not in verticeAtual.restricoes):
                if(i in verticeAtual.preferencias and i not in grafo.preferenciasAtendidas):
                    grafo.preferenciasAtendidas.append(i)
                    verticeAtual.preferenciasNaoAtendidas -= 1
                verticeAtual.cor = i
                if(i not in cores):
                    cores.append(i)
                break
        verticeAtual = grauSaturacao(grafo, verticeAtual)
        
    grafo.cores = len(cores)
    return len(cores)

A outra variação do algoritmo DSatur prioriza a coloração das preferências, de forma a não violar as restrições, tenta-se 
colorir com uma das cores preferenciais, caso tenha sucesso é contabilizado nas preferências atendidas, caso não seja possível
colorir o vértice com essa cor tem-se uma nova tentativa na coloração com as demais cores (horários) disponíveis.

In [ ]:
def DSATURpreferencial(grafo):
    listaAdj = grafo.listaAdj
    vertices = grafo.vertices
    horarios = grafo.horarios
    flagPreferencia = False
    cores = []
    verticeAtual = grafo.getInicial()
    for v in listaAdj:
        coresVizinhos = []
        for vizinhos in grafo.getVizinhos(verticeAtual.ID):
            if(vertices[vizinhos].cor != -1 and vertices[vizinhos].cor not in coresVizinhos):
                coresVizinhos.append(vertices[vizinhos].cor)
        for corPreferencial in verticeAtual.preferencias:
            if(corPreferencial not in coresVizinhos and corPreferencial not in verticeAtual.restricoes):
                verticeAtual.cor = corPreferencial
                verticeAtual.preferenciasNaoAtendidas -= 1
                if(corPreferencial not in grafo.preferenciasAtendidas):
                    grafo.preferenciasAtendidas.append(corPreferencial)
                flagPreferencia = True
                if(corPreferencial not in cores):
                    cores.append(corPreferencial)
        if(flagPreferencia == False):
            for i in range (horarios*5):
                if(i not in coresVizinhos and i not in verticeAtual.restricoes):
                    verticeAtual.cor = i
                    if(i not in cores):
                        cores.append(i)
                    break
        flagPreferencia = False
        verticeAtual = grauSaturacao(grafo, verticeAtual)
    
    grafo.cores = len(cores)
    return len(cores)

Para a seleção de ambos os algoritmos foi usado o critério do próprio algoritmo DSatur, que é o grau de saturação, segue o
mesmo padrão do algoritmo original pegando o vértice de maior grau de saturação e em caso de empate o eleito será o de maior
grau.

In [ ]:
def grauSaturacao(grafo, atual):
    maxDeegre = 0
    verticeCandidato = []
    lista = []
    for v in grafo.listaAdj:
        if(grafo.vertices[v].cor == -1):
            for vizinho in grafo.getVizinhos(v):
                if(grafo.vertices[vizinho].cor != -1 and grafo.vertices[vizinho].cor not in lista):
                    lista.append(grafo.vertices[vizinho].cor)
            if(len(lista) > maxDeegre):
                maxDeegre = len(lista)
                verticeCandidato = []
                verticeCandidato.append(grafo.vertices[v])
            elif(len(lista) == maxDeegre):
                verticeCandidato.append(grafo.vertices[v])
        lista = []
    if(len(verticeCandidato) == 1):
        return verticeCandidato[0]
    elif(len(verticeCandidato) == 0):
            return None
    else:
        grauMaior = 0
        posicao = 0
        for i in range (len(verticeCandidato)):
            if(verticeCandidato[i].grau > grauMaior):
                grauMaior = verticeCandidato[i].grau
                posicao = i
        return verticeCandidato[posicao]

Com a explicação de como foi feito para encontrar a solução, agora será visto como montar o grafo a partir de um arquivo excel
que é um tipo de arquivo muito utilizado para esse tipo de problema de horários de escolas. A leitura é feita seis passos, o primeiro é a obtenção dos vértices do grafo, fazendo a criação de todos os vértices e atribuindo seus dados, o segundo passo é a criação das arestas, explicadas anteriormente, o retorno do método indica se será criada ou não a aresta, o terceiro passo tem-se a leitura dos horários disponíveis de aula e é gerado as possíveis cores para cada dia, o quarto, quinto e sexto passo são para a alocação das restrições de turma, restrições de professores e preferências de professores, respectivamente. Retorna o grafo resultante 

In [ ]:
def leGrafo(nomeExcel):
    grafo = ListaAdjacencia()

    '''Lendo os dados do grafo'''
    planilha = pd.read_excel(nomeExcel, sheet_name=0)
    colunas = planilha.columns
    qtdLinhas = int(planilha.index.stop)
    for linha in range (qtdLinhas):
        for aulas in range (int(planilha[colunas[3]][linha])):
            grafo.adicionaVertice(planilha[colunas[2]][linha], planilha[colunas[0]][linha], planilha[colunas[1]][linha])
    
    '''Fazendo ligacoes de conflito'''
    for i in range (grafo.numeroVertices()):
        for j in range (grafo.numeroVertices()):
            if(i != j and grafo.verificaConflito(i, j) == True):
                grafo.adicionaAresta(i, j)
    grafo.ajustaGrau()

    '''Lendo configuracoes'''
    sheetConfig = pd.read_excel(nomeExcel, sheet_name = 1)
    configuracao = []
    sheetLinhas = int(sheetConfig.index.stop)
    sheetColunas = sheetConfig.columns
    for i in range (0, sheetLinhas):
       configuracao.append(str(sheetConfig["Horários de Inicio de aulas a cada dia."][i]))

    '''Criar matriz de cores'''
    grafo.criaListaCores(configuracao)

    '''Lendo restricoes da turma'''
    restricoesTurma = pd.read_excel(nomeExcel, sheet_name = 3)
    if(not restricoesTurma.empty):
        grafo.restricoesTurma(restricoesTurma)

    '''Lendo restricoes dos professores'''
    restricoesProfessor = pd.read_excel(nomeExcel, sheet_name = 2)
    if(not restricoesProfessor.empty):
        grafo.restricoesProfessor(restricoesProfessor)
    
    '''Lendo preferencias dos professores'''
    preferenciasProfessor = pd.read_excel(nomeExcel, sheet_name = 4)
    if(not preferenciasProfessor.empty):
        grafo.preferenciasProfessor(preferenciasProfessor)
    
    return grafo

Após a criação e configuração do grafo o algoritmo de coloração é executado nas suas duas formas, com priorização e sem 
priorização das preferências, após a execução é verificado qual se comportou melhor, olhando os resultados.

In [ ]:
def obterResultado(grafo):
    '''Executando DSATUR com preferencia'''
    totalCoresPref = DSATURpreferencial(grafo)
    totalNaoColoridosPref = grafo.verificaColoracao()
    totalPrefAtendidasPref = grafo.verificaPreferencias()

    '''Resetando as cores'''
    grafo.resetar()

    '''Executando DSATUR sem preferencia'''
    totalCoresSemPref = DSATUR(grafo)
    totalNaoColoridosSemPref = grafo.verificaColoracao()
    totalPrefAtendidasSemPref = grafo.verificaPreferencias()

    '''Comparando resultados'''
    if(totalCoresPref < totalCoresSemPref):
        grafo.cores = str(totalCoresPref)
        grafo.naoColoridos = str(totalNaoColoridosPref)
        print("Quantidade de cores: " + str(totalCoresPref))
        print("Vértices não coloridos: " + str(totalNaoColoridosPref))
        print("Preferencias atendidas sobre o total de preferências: " + str(totalPrefAtendidasPref))
    else:
        if(totalNaoColoridosPref < totalNaoColoridosSemPref):
            grafo.cores = str(totalCoresPref)
            grafo.naoColoridos = str(totalNaoColoridosPref)
            print("Quantidade de cores: " + str(totalCoresPref))
            print("Vértices não coloridos: " + str(totalNaoColoridosPref))
            print("Preferencias atendidas sobre o total de preferências: " + str(totalPrefAtendidasPref)) 
        elif(totalNaoColoridosPref == totalNaoColoridosSemPref):
            if(totalPrefAtendidasPref > totalPrefAtendidasSemPref):
                grafo.cores = str(totalCoresPref)
                grafo.naoColoridos = str(totalNaoColoridosPref)
                print("Quantidade de cores: " + str(totalCoresPref))
                print("Vértices não coloridos: " + str(totalNaoColoridosPref))
                print("Preferencias atendidas sobre o total de preferências: " + str(totalPrefAtendidasPref))
            else:
                grafo.cores = str(totalCoresSemPref)
                grafo.naoColoridos = str(totalNaoColoridosSemPref)
                print("Quantidade de cores: " + str(totalCoresSemPref))
                print("Vértices não coloridos: " + str(totalNaoColoridosSemPref))
                print("Preferencias atendidas sobre o total de preferências: " + str(totalPrefAtendidasSemPref)) 
        else:
            grafo.cores = str(totalCoresSemPref)
            grafo.naoColoridos = str(totalNaoColoridosSemPref)
            print("Quantidade de cores: " + str(totalCoresSemPref))
            print("Vértices não coloridos: " + str(totalNaoColoridosSemPref))
            print("Preferencias atendidas sobre o total de preferências: " + str(totalPrefAtendidasSemPref)) 

Para cobrir a quantidade de escolas é criado um grafo por escola, seguindo todo o padrão exibido anteriormente, e exibindo,
o tempo gasto para cada escola.

In [ ]:
def main():

    print("Escola A:")
    escolaA = leGrafo("Escola_A.xlsx")
    inicio = timeit.default_timer()
    obterResultado(escolaA)
    fim = timeit.default_timer()
    tempoA = fim - inicio
    print()

    print("Escola B:")
    escolaB = leGrafo("Escola_B.xlsx")
    inicio = timeit.default_timer()
    obterResultado(escolaB)
    fim = timeit.default_timer()
    tempoB = fim - inicio
    print()

    print("Escola C:")
    escolaC = leGrafo("Escola_C.xlsx")
    inicio = timeit.default_timer()
    obterResultado(escolaC)
    fim = timeit.default_timer()
    tempoC = fim - inicio
    print()

    print("Escola D:")
    escolaD = leGrafo("Escola_D.xlsx")
    inicio = timeit.default_timer()
    obterResultado(escolaD)
    fim = timeit.default_timer()
    tempoD = fim - inicio

# Conclusão

Com esse trabalho foi possível observar que o algoritmo de DSatur em suas duas variações utilizadas, se comporta de maneira
razoável, tendo em vista que é heurístico, o motivo da utilização dessas duas abordagens do algoritmo está relacionado com 
o desempenho na questão de vértices não coloridos, na escola A e D teve-se uma redução no número de vértices não coloridos utilizando
o algoritmo com prioridade, porém o mesmo se comportou mal no ambiente da escola B aumentando o número de vértices não coloridos
e na escola C o desempenho foi igual. O código como um todo pega o melhor resultado que obtiver, independente do algoritmo.
O critério para escolha é uma hierarquia de menor quantidade de cor, menor quantidade de vértices não coloridos e por fim
maior quantidade de preferências atendidas.